In [13]:
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)
from facenet_pytorch import InceptionResnetV1
import cv2
import torch
from insightface.app import FaceAnalysis
from insightface.utils import face_align
import torch.nn.functional as F
from deep_sort_realtime.deepsort_tracker import DeepSort
from collections import defaultdict, Counter
import json
from urllib.parse import quote
import subprocess
import numpy as np
from torchvision import transforms
from PIL import Image
import os


In [9]:
app = FaceAnalysis(providers=["CUDAExecutionProvider", "CPUExecutionProvider"])
app.prepare(ctx_id=0, det_size=(640, 640))

download_path: /home/scifre/.insightface/models/buffalo_l


100%|██████████| 281857/281857 [00:14<00:00, 19945.47KB/s]



Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}, 'CUDAExecutionProvider': {'prefer_nhwc': '0', 'enable_skip_layer_norm_strict_mode': '0', 'tunable_op_enable': '0', 'enable_cuda_graph': '0', 'tunable_op_max_tuning_duration_ms': '0', 'tunable_op_tuning_enable': '0', 'cudnn_conv_use_max_workspace': '1', 'use_tf32': '1', 'cudnn_conv1d_pad_to_nc1d': '0', 'do_copy_in_default_stream': '1', 'cudnn_conv_algo_search': 'EXHAUSTIVE', 'gpu_external_empty_cache': '0', 'gpu_external_free': '0', 'gpu_external_alloc': '0', 'gpu_mem_limit': '18446744073709551615', 'arena_extend_strategy': 'kNextPowerOfTwo', 'user_compute_stream': '0', 'has_user_compute_stream': '0', 'use_ep_level_unified_stream': '0', 'device_id': '0'}}
find model: /home/scifre/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CPUExecutionProvider': {},

In [10]:
model = InceptionResnetV1(pretrained="vggface2").eval().to("cuda")

In [16]:
transform = transforms.Compose([
    transforms.ToTensor(),  # Converts PIL/numpy to tensor and scales to [0,1]
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Scales to [-1,1]
    ])


In [6]:
import lancedb
import pyarrow as pa


In [4]:
lancedb.connect("face_db")

LanceDBConnection(uri='/home/scifre/Desktop/projects/Minor-7/face_db')

In [7]:
schema = pa.schema([
    ("label", pa.string()),
    ("embedding", pa.list_(pa.float32(), 512))
    ])


In [8]:
import lancedb
db = lancedb.connect("face_db")
try:
    table = db.create_table("face_data", schema=schema)
    print("Created table 'face_data'")
except Exception as e:
    # If table already exists, open it
    try:
        table = db.get_table("face_data")
        print("Opened existing table 'face_data'")
    except Exception:
        raise


Created table 'face_data'


In [11]:
face_table = db.open_table("face_data")

In [17]:
for file in os.listdir("face_images"):
    if file.endswith(".jpeg"):
        img = cv2.imread(os.path.join("face_images", file))
        
        face = app.get(img)[0]

        aligned_face = face_align.norm_crop(img, face.kps, image_size=224)
        aligned_face = cv2.resize(aligned_face, (160, 160))
        aligned_face = cv2.cvtColor(aligned_face, cv2.COLOR_BGR2RGB)
        
        face_tensor = transform(aligned_face).unsqueeze(0)

        embedding = model(face_tensor.to("cuda"))
        embedding = F.normalize(embedding, p=2, dim=1).cpu().detach().numpy()  
        flattened_emb = np.asarray(embedding).flatten()
        face_table.add([{
            "label": file,
            "embedding": flattened_emb
        }])


In [20]:
import pandas as pd
df = face_table.to_pandas()
df.shape

(8, 2)

In [21]:
video_path = "VID_20250915_155804628.mp4"

cap = cv2.VideoCapture(video_path)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    faces = app.get(frame)

    for face in faces:
        box = face.bbox.astype(int)
        cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 2)

    cv2.imshow('Video', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

: 